In [4]:
!pip install wget
!pip install Sastrawi
!pip install nlp_id
!pip install unidecode

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=9b23c05929f6217e58690e712856276f6675c227fc46bf63ce353abeb69cb235
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 32.7 MB/s eta 0:00:00
  Created wheel for nlp_id: filename=nlp_id-0.1.15.0-py3-none-any.whl size=58153891 sha256=a4f15c1bed565db8fe29ac324528d1a5e186a0136c698b9d868f1aa69334691f
  Stored in directory: /root/.cache/pip/wheels/85/34/2a/deaf7b7896a1eed336b874e1a7732588d40bb7bdc0a824e73e
Successfully built nlp_id
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.3
    Uninstalling pytest-7.4.3:
  

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import re
import os
import pickle
import warnings
import wget
import sklearn
import random
import nltk
import spacy
import textblob
import gensim
import Sastrawi
from gensim.models import Word2Vec
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, GRU, SimpleRNN
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
# from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from nltk.corpus import stopwords
from nltk.tree import Tree
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter, OrderedDict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from spacy.util import minibatch, compounding
from spacy import load, displacy
from spacy.training.example import Example
from nlp_id import stopword
from unidecode import unidecode
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
df = pd.read_csv('./sample_data/tweet.csv')

In [7]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1815 non-null   int64 
 1   sentimen    1815 non-null   object
 2   tweet       1815 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.7+ KB


,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...


In [8]:
tweet = df['tweet']
display(tweet[0])
display(tweet)

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

0       Kata @prabowo Indonesia tidak dihargai bangsa ...
1       Batuan Langka, Tasbih Jokowi Hadiah dari Habib...
2       Di era Jokowi, ekonomi Indonesia semakin baik....
3       Bagi Sumatera Selatan, Asian Games berdampak p...
4       Negara kita ngutang buat bngun infrastruktur y...
                              ...                        
1810    Negarawan sejati sll bangga dan mengedepankan ...
1811    1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812    Mari bangun bangsa dgn mendukung perekonomian ...
1813    Bantu majukan perekonomian bangsa bersama Pak ...
1814    Pak @jokowi mengubah cara pandang ekonomi. Kin...
Name: tweet, Length: 1815, dtype: object

## Text Cleaning

In [9]:
tweet_cleaned = []

  # Looping setiap kalimat
for sentence in tweet:
    # Hapus hashtag
    sentence = re.sub(r"#\w+", "", sentence)

    # hapus @
    sentence = sentence.replace('@', '')

    # hspus pic.twitter.com
    sentence = re.sub(r"(pic.twitter.com/[^\s]+)", '', sentence)

    # Hapus karakter non-alfanumerik
    # sentence = re.sub(r"[^\w\s]+", "", sentence)

    # Konversi ke representasi ASCII
    # sentence = unidecode(sentence)

    # Hapus spasi ekstra
    sentence = re.sub(r"\s+", " ", sentence)

    # Trim kalimat
    sentence = sentence.strip()

    # Tambahkan kalimat baru ke dalam list
    tweet_cleaned.append(sentence)

display(tweet[0])
print('-'*50)
display(tweet_cleaned[0])

'Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦'

--------------------------------------------------


'Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â â€¦'

### Remove URL LINK

In [10]:
pattern = r"(https?://.[^\s]+)"
tweet_no_url = [re.sub(pattern, "", sentence) for sentence in tweet_cleaned]

display(tweet_cleaned[0])
print('-'*50)
display(tweet_no_url[0])

'Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â â€¦'

--------------------------------------------------


'Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ  â€¦'

### Remove Punctuation & Special Character

In [11]:
def remove_punctuation(sentences):
  """Menghilangkan punctuation dari list kalimat."""

  # Buat regular expression untuk punctuation
  regex = re.compile(r"[^a-zA-Z0-9]")

  # Hapus punctuation dari setiap kalimat
  for sentence in sentences:
    sentence = regex.sub("", sentence)

  return sentences

def remove_punctuation_and_special_character(text):
  # clean puntuation and special character using regex
  cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  return cleaned_text

In [12]:
# regex = re.compile(r"[^a-zA-Z0-9]")

tweet_no_punct = []
for idx in range(len(tweet_no_url)):
  cleaned = remove_punctuation_and_special_character(tweet_no_url[idx])
  tweet_no_punct.append(cleaned)

print(tweet_no_url[0])
print(tweet_no_punct[0])


Kata prabowo Indonesia tidak dihargai bangsa asing! Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan rockygerung?ðŸ˜œ  â€¦
Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini  pasti hoax buatan penguasa ya kan rockygerung  


### Remove Double Space

In [13]:
tweet_cleaned_double_space = []
def remove_double_space(text):
    cleaned_text = re.sub(r"\s+", " ", text)
    return cleaned_text

for index in range(len(tweet_no_punct)):
    text = remove_double_space(tweet_no_punct[index])
    tweet_cleaned_double_space.append(text)

print(tweet_no_punct[0])
print(tweet_cleaned_double_space[0])

Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini  pasti hoax buatan penguasa ya kan rockygerung  
Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini pasti hoax buatan penguasa ya kan rockygerung 


### Remove Stopwords

In [14]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stop_factory = StopWordRemoverFactory()
# more_stopword = [‘dengan’, ‘ia’,’bahwa’,’oleh’]
# data = stop_factory.get_stop_words()+more_stopword
data = stop_factory.get_stop_words()
stopword_remover = stop_factory.create_stop_word_remover()
print(data)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [15]:
raw_stopwords = ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', 'oh', 'ok', 'seharusnya', 'sebetulnya', 'setiap', 'setidaknya', 'sesuatu', 'pasti', 'saja', 'toh', 'walau', 'tolong', 'tentu', 'amat', 'apalagi', 'bagaimanapun']

In [16]:
def remove_stopwords(text):
    # Kode untuk menghapus stopwords
    stop_words = set(stopwords.words('indonesian'))
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

def remove_stopwords_sastrawi(text):
    # Kode untuk menghapus stopwords
    text_lower = text.lower()
    stop_words = set(raw_stopwords)
    tokens = word_tokenize(text_lower)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

In [17]:
tweet_no_stopwords = []
for index in range(len(tweet_cleaned_double_space)):
    cleaned_text = remove_stopwords_sastrawi(tweet_cleaned_double_space[index])
    tweet_no_stopwords.append(cleaned_text)

display(tweet_cleaned_double_space[0])
display(tweet_no_stopwords[0])

'Kata prabowo Indonesia tidak dihargai bangsa asing Berita ini pasti hoax buatan penguasa ya kan rockygerung '

'kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung'

In [18]:
display(tweet_cleaned_double_space[2])
display(tweet_no_stopwords[2])
print('--'*50)
display(tweet_cleaned_double_space[3])
display(tweet_no_stopwords[3])
print('--'*50)
display(tweet_cleaned_double_space[4])
display(tweet_no_stopwords[4])
print('--'*50)
display(tweet_cleaned_double_space[5])
display(tweet_no_stopwords[5])
print('--'*50)

'Di era Jokowi ekonomi Indonesia semakin baik'

'era jokowi ekonomi indonesia semakin baik'

----------------------------------------------------------------------------------------------------


'Bagi Sumatera Selatan Asian Games berdampak pd ekonomi langsung diprediksi mencapai 185 triliun Indonesia maju Jokowi hebat'

'sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai 185 triliun indonesia maju jokowi hebat'

...

'negara ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakatnya ngeluh negara ngutang negara ngutang utang dibayar kalo negara dapet penghasilan penghasilan negara ya pajak'

----------------------------------------------------------------------------------------------------


'Yg bisikin pak jokowi cm mikirin perputaran duit di golongan tertentu ESports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia bukan hal fundamental Betul yg dikatakan prabowo walau dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi'

'yg bisikin pak jokowi cm mikirin perputaran duit golongan tertentu esports sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan fundamental betul yg dikatakan prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi'

----------------------------------------------------------------------------------------------------


In [19]:
tweet_no_stopwords

['kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy',
 'era jokowi ekonomi indonesia semakin baik',
 'sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai 185 triliun indonesia maju jokowi hebat',
 'negara ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakatnya ngeluh negara ngutang negara ngutang utang dibayar kalo negara dapet penghasilan penghasilan negara ya pajak',
 'yg bisikin pak jokowi cm mikirin perputaran duit golongan tertentu esports sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan fundamental betul yg dikatakan prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi',
 'masa tenang msih ngoceh ajattp jokowi harga mati',
 'kerjasa ekonomi bilateral multilateral doakan tps dengarkan suara hati',
 'iya bener aa mantap kan pilihan pemimpin yg bs memberi solusi jokowi bukan yg meny

### Replace Slang

In [20]:
# def replace_slang(text):
#     slang_model = spacy.load("id_core_web_lg")
#     slang_doc = slang_model(text)
#     slang_words = [token for token in slang_doc if token.pos == "ADJ"]
# def replace_slang_with_spacy(text, slang_dict):
#     nlp = spacy.load("id_core_web_md")
#     doc = nlp(text)
#     replaced_tokens = []

#     for token in doc:
#         if token.text in slang_dict:
#             replaced_tokens.append(slang_dict[token.text])
#         else:
#             replaced_tokens.append(token.text)

#     return " ".join(replaced_tokens)


def replace_slang_word(doc,slang_word):
    for index in  range(0,len(doc)-1):
        index_slang = slang_word.slang==doc[index]
        formal = list(set(slang_word[index_slang].formal))
        # if len(formal)==1:
        #     doc[index]=formal[0]
    return doc



In [21]:
indo_slang_dict = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")
indo_slang_dict.head()

,slang,formal,In-dictionary,context,category1,category2,category3
0,woww,wow,1,wow,elongasi,0,0
1,aminn,amin,1,Selamat ulang tahun kakak tulus semoga panjang...,elongasi,0,0
2,met,selamat,1,Met hari netaas kak!? Wish you all the best @t...,abreviasi,0,0
3,netaas,menetas,1,Met hari netaas kak!? Wish you all the best @t...,afiksasi,elongasi,0
4,keberpa,keberapa,0,Birthday yg keberpa kak?,abreviasi,0,0


In [22]:

# slang_dict.read()
tweet_no_slang = []
for index in range(len(tweet_no_stopwords)):
    new = replace_slang_word(tweet_no_stopwords[index], indo_slang_dict)
    tweet_no_slang.append(new)

display(tweet_no_stopwords[0])
display(tweet_no_slang)

'kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung'

['kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung',
 'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy',
 'era jokowi ekonomi indonesia semakin baik',
 'sumatera selatan asian games berdampak pd ekonomi langsung diprediksi mencapai 185 triliun indonesia maju jokowi hebat',
 'negara ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakatnya ngeluh negara ngutang negara ngutang utang dibayar kalo negara dapet penghasilan penghasilan negara ya pajak',
 'yg bisikin pak jokowi cm mikirin perputaran duit golongan tertentu esports sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan fundamental betul yg dikatakan prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi',
 'masa tenang msih ngoceh ajattp jokowi harga mati',
 'kerjasa ekonomi bilateral multilateral doakan tps dengarkan suara hati',
 'iya bener aa mantap kan pilihan pemimpin yg bs memberi solusi jokowi bukan yg meny

In [23]:
display(tweet_no_stopwords[0])
display(tweet_no_slang[0])

'kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung'

'kata prabowo indonesia tidak dihargai bangsa asing berita hoax buatan penguasa ya kan rockygerung'

In [24]:
display(tweet_no_stopwords[1])
display(tweet_no_slang[1])
print('--'*50)
display(tweet_no_stopwords[2])
display(tweet_no_slang[2])
print('--'*50)
display(tweet_no_stopwords[3])
display(tweet_no_slang[3])
print('--'*50)
display(tweet_no_stopwords[4])
display(tweet_no_slang[4])
print('--'*50)
display(tweet_no_stopwords[5])
display(tweet_no_slang[5])
print('--'*50)

'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy'

'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy'

----------------------------------------------------------------------------------------------------


'era jokowi ekonomi indonesia semakin baik'

'era jokowi ekonomi indonesia semakin baik'

...

'negara ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakatnya ngeluh negara ngutang negara ngutang utang dibayar kalo negara dapet penghasilan penghasilan negara ya pajak'

----------------------------------------------------------------------------------------------------


'yg bisikin pak jokowi cm mikirin perputaran duit golongan tertentu esports sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan fundamental betul yg dikatakan prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi'

'yg bisikin pak jokowi cm mikirin perputaran duit golongan tertentu esports sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan fundamental betul yg dikatakan prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi'

----------------------------------------------------------------------------------------------------


## Normalization

### Stemming

In [25]:
def normalization_stemming(text):
  # create an instance of the stemmer
  stemmer = StemmerFactory().create_stemmer()

  # perform stemming
#   cleaned_text = stemmer.stem(text)
  cleaned_text = stemmer.stem(text)
  return cleaned_text

In [26]:
normalized_stemming = []
for idx in range(len(tweet_no_stopwords)):
    normalized = normalization_stemming(tweet_no_stopwords[idx])
    normalized_stemming.append(normalized)

display(tweet_no_stopwords[1])
display(normalized_stemming[1])

'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy'

'batu langka tasbih jokowi hadiah habib luthfi harga mercy'

In [27]:
display(tweet_no_stopwords[1])
display(normalized_stemming[1])
print('--'*30)
display(tweet_no_stopwords[2])
display(normalized_stemming[2])
print('--'*30)
display(tweet_no_stopwords[3])
display(normalized_stemming[3])
print('--'*30)
display(tweet_no_stopwords[4])
display(normalized_stemming[4])
print('--'*30)
display(tweet_no_stopwords[5])
display(normalized_stemming[5])
print('--'*30)

'batuan langka tasbih jokowi hadiah habib luthfi seharga mercy'

'batu langka tasbih jokowi hadiah habib luthfi harga mercy'

------------------------------------------------------------


'era jokowi ekonomi indonesia semakin baik'

'era jokowi ekonomi indonesia makin baik'

...

'negara ngutang buat bngun infrastruktur udah dipake masyarakat terus masyarakat ngeluh negara ngutang negara ngutang utang bayar kalo negara dapet hasil hasil negara ya pajak'

------------------------------------------------------------


'yg bisikin pak jokowi cm mikirin perputaran duit golongan tertentu esports sebagian kecil dr sekian besar penentu pertumbuhan perekonomian indonesia bukan fundamental betul yg dikatakan prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi'

'yg bisikin pak jokowi cm mikirin putar duit golong tentu esports bagi kecil dr sekian besar tentu tumbuh ekonomi indonesia bukan fundamental betul yg kata prabowo dgn bhs yg sederhana tp mrpkan aspek fundamental ekonomi'

------------------------------------------------------------


In [30]:
normalized_stemming_no_slang = []
# for idx in range(len(tweet_no_slang)):
#     normalized = normalization_stemming(tweet_no_slang[idx])
#     normalized_stemming_no_slang.append(normalized)

# display(tweet_no_slang[1])
# display(normalized_stemming_no_slang[1])

In [31]:
len(normalized_stemming_no_slang)

0

In [32]:
def normalization_lemmatizing(text):
  lemmatizer = WordNetLemmatizer()
  lemmatized = lemmatizer.lemmatize(text)
  return lemmatized

In [33]:
# normalized_lemmatizing = []
# for idx in range(len(tweet_no_stopwords)):
#     new = normalization_lemmatizing(tweet_no_stopwords[idx])
#     normalized_lemmatizing.append(new)

# display(tweet_no_stopwords[1])
# display(normalized_lemmatizing[1])

In [34]:
# tweet_no_slang

In [35]:
# normalized_stemming_no_slang

# Tokenization

In [36]:
def word_tokenization(text):
  tokens = word_tokenize(text)
  return tokens

In [38]:
tweet_w_tokens = []

for idx in range(len(normalized_stemming)):
  tokens = word_tokenization(normalized_stemming[idx])
  tweet_w_tokens.append(tokens)

display(normalized_stemming[0])
display(tweet_w_tokens[0])

'kata prabowo indonesia tidak harga bangsa asing berita hoax buat kuasa ya kan rockygerung'

['kata',
 'prabowo',
 'indonesia',
 'tidak',
 'harga',
 'bangsa',
 'asing',
 'berita',
 'hoax',
 'buat',
 'kuasa',
 'ya',
 'kan',
 'rockygerung']

In [39]:
tweet_w_tokens

[['kata',
  'prabowo',
  'indonesia',
  'tidak',
  'harga',
  'bangsa',
  'asing',
  'berita',
  'hoax',
  'buat',
  'kuasa',
  'ya',
  'kan',
  'rockygerung'],
 ['batu',
  'langka',
  'tasbih',
  'jokowi',
  'hadiah',
  'habib',
  'luthfi',
  'harga',
  'mercy'],
 ['era', 'jokowi', 'ekonomi', 'indonesia', 'makin', 'baik'],
 ['sumatera',
  'selatan',
  'asi',
  'games',
  'dampak',
  'pd',
  'ekonomi',
  'langsung',
  'prediksi',
  'capai',
  '185',
  'triliun',
  'indonesia',
  'maju',
  'jokowi',
  'hebat'],
 ['negara',
  'ngutang',
  'buat',
  'bngun',
  'infrastruktur',
  'udah',
  'dipake',
  'masyarakat',
  'terus',
  'masyarakat',
  'ngeluh',
  'negara',
  'ngutang',
  'negara',
  'ngutang',
  'utang',
  'bayar',
  'kalo',
  'negara',
  'dapet',
  'hasil',
  'hasil',
  'negara',
  'ya',
  'pajak'],
 ['yg',
  'bisikin',
  'pak',
  'jokowi',
  'cm',
  'mikirin',
  'putar',
  'duit',
  'golong',
  'tentu',
  'esports',
  'bagi',
  'kecil',
  'dr',
  'sekian',
  'besar',
  'tentu',


# Vectorization

In [49]:
 normalized_array_data = np.array(normalized_stemming)
 normalized_array_data
 display(normalized_array_data.shape)

(1815,)

In [50]:
vectorizer = CountVectorizer()
vectorizer_tfidf = TfidfVectorizer()

In [51]:
# tweet_vectorized = []

# for idx in range(len(normalized_array_data)):
#   vectorized = vectorizer.fit_transform(normalized_array_data[idx])
#   tweet_vectorized.append(vectorized)

tweet_vectorized = vectorizer.fit_transform(normalized_array_data)
# tweet_vectorized = vectorizer.fit(normalized_array_data)

display(normalized_array_data[1])
display(tweet_vectorized[1])

'batu langka tasbih jokowi hadiah habib luthfi harga mercy'

<1x5816 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [52]:
tweet_vectorized

<1815x5816 sparse matrix of type '<class 'numpy.int64'>'
	with 31791 stored elements in Compressed Sparse Row format>

In [53]:
print(tweet_vectorized)

  (0, 2467)	1
  (0, 4255)	1
  (0, 2059)	1
  (0, 5359)	1
  (0, 1888)	1
  (0, 547)	1
  (0, 435)	1
  (0, 699)	1
  (0, 1963)	1
  (0, 902)	1
  (0, 2796)	1
  (0, 5753)	1
  (0, 2424)	1
  (0, 4528)	1
  (1, 1888)	1
  (1, 594)	1
  (1, 2875)	1
  (1, 5219)	1
  (1, 2317)	1
  (1, 1852)	1
  (1, 1842)	1
  (1, 3023)	1
  (1, 3300)	1
  (2, 2059)	1
  (2, 2317)	1
  :	:
  (1812, 554)	1
  (1812, 478)	1
  (1813, 547)	1
  (1813, 2317)	1
  (1813, 1469)	1
  (1813, 3061)	1
  (1813, 3902)	1
  (1813, 5800)	1
  (1813, 4599)	1
  (1813, 560)	1
  (1814, 2059)	1
  (1814, 2317)	1
  (1814, 1469)	1
  (1814, 3902)	1
  (1814, 5548)	1
  (1814, 2186)	1
  (1814, 3849)	1
  (1814, 4795)	1
  (1814, 2907)	1
  (1814, 1986)	1
  (1814, 973)	1
  (1814, 3938)	1
  (1814, 4323)	1
  (1814, 2636)	1
  (1814, 1457)	1


In [54]:
tweet_vectorized_array = tweet_vectorized.toarray()
# tweet_vectorized = np.array(tweet_vectorized)
new_df = pd.DataFrame(data=tweet_vectorized_array, columns= vectorizer.get_feature_names_out())
new_df

,01,01bpk,02,02pak,02petaninelayanpeternak,08,0850,10,100,1000,...,yuuk,zaenal,zakat,zaman,zer,zero,ziarah,zippo,zon,zonk
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1811,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1812,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
new_df.shape

(1815, 5816)

In [56]:
new_df[1:]

,01,01bpk,02,02pak,02petaninelayanpeternak,08,0850,10,100,1000,...,yuuk,zaenal,zakat,zaman,zer,zero,ziarah,zippo,zon,zonk
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1811,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1812,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Split Data

In [57]:
# df = pd.read_csv('./sample_data/tweet.csv')

In [58]:
print(df['sentimen'].value_counts)
print(df.head())
display(df)

<bound method IndexOpsMixin.value_counts of 0       negatif
1        netral
2        netral
3       positif
4       negatif
         ...   
1810     netral
1811     netral
1812     netral
1813     netral
1814     netral
Name: sentimen, Length: 1815, dtype: object>
   Unnamed: 0 sentimen                                              tweet
0           0  negatif  Kata @prabowo Indonesia tidak dihargai bangsa ...
1           1   netral  Batuan Langka, Tasbih Jokowi Hadiah dari Habib...
2           2   netral  Di era Jokowi, ekonomi Indonesia semakin baik....
3           3  positif  Bagi Sumatera Selatan, Asian Games berdampak p...
4           4  negatif  Negara kita ngutang buat bngun infrastruktur y...


,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...
...,...,...,...
1810,1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...


In [59]:
mapping = {
    "negatif": 0,
    "netral": 1,
    "positif": 1
}

df['sentimen'] = [mapping[label] for label in df['sentimen']]
target = df['sentimen']


In [62]:
X = pd.DataFrame(normalized_stemming, columns=['tweet'])
Y = target

display(X)
display(Y)

,tweet
0,kata prabowo indonesia tidak harga bangsa asin...
1,batu langka tasbih jokowi hadiah habib luthfi ...
2,era jokowi ekonomi indonesia makin baik
3,sumatera selatan asi games dampak pd ekonomi l...
4,negara ngutang buat bngun infrastruktur udah d...
...,...
1810,negarawan sejati sll bangga depan harga diri b...
1811,1 hrs ceramah damai indonesia 2 ekonomi makin ...
1812,bangun bangsa dgn dukung ekonomi negara sama p...
1813,bantu maju ekonomi bangsa sama pak jokowi yuk


0       0
1       1
2       1
3       1
4       0
       ..
1810    1
1811    1
1812    1
1813    1
1814    1
Name: sentimen, Length: 1815, dtype: int64

In [63]:
display(X)

,tweet
0,kata prabowo indonesia tidak harga bangsa asin...
1,batu langka tasbih jokowi hadiah habib luthfi ...
2,era jokowi ekonomi indonesia makin baik
3,sumatera selatan asi games dampak pd ekonomi l...
4,negara ngutang buat bngun infrastruktur udah d...
...,...
1810,negarawan sejati sll bangga depan harga diri b...
1811,1 hrs ceramah damai indonesia 2 ekonomi makin ...
1812,bangun bangsa dgn dukung ekonomi negara sama p...
1813,bantu maju ekonomi bangsa sama pak jokowi yuk


In [64]:
X_train, X_test, Y_train, Y_test = train_test_split(X.tweet, Y, test_size=0.3, random_state=0)

In [65]:
from keras.utils import pad_sequences

In [66]:
# X_train = [str(text) for text in X_train if isinstance(text, str)]
X_train = [str(text).lower() if isinstance(text, str) else str(text) for text in X_train]
max_words = 1000
max_len = 150
tok = Tokenizer()
tok.fit_on_texts(X_train)
sequence = tok.texts_to_sequences(X_train)
sequence_matrix = pad_sequences(sequence, maxlen=max_len)

In [67]:
sequence_matrix

array([[   0,    0,    0, ...,  660, 1963, 1335],
       [   0,    0,    0, ...,  562,  198,  162],
       [   0,    0,    0, ...,  171,  144, 1968],
       ...,
       [   0,    0,    0, ..., 1945,  414,  881],
       [   0,    0,    0, ...,   26, 4741, 1819],
       [   0,    0,    0, ...,  249,   55,    7]], dtype=int32)

In [68]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 64, input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model

In [69]:
from keras import backend as K
from keras.optimizers.legacy import Adam

In [93]:

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

model = RNN()
model.summary()
# model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', 'precission', 'recall', 'f1-score'])
# model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', precision, recall, f1_score])
# model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', 'AUC'])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 20)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 20, 64)            64000     
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

In [94]:
model.fit(sequence_matrix, Y_train,batch_size=128, epochs=10, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10


InvalidArgumentError: ignored

In [72]:
len(sequence_matrix)

1270

In [73]:
len(Y_train)

1270

In [95]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Tokenization and padding
max_words = 1000  # Maximum number of words in the vocabulary
max_len = 20  # Maximum length of a tweet

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequence = tok.texts_to_sequences(X_train)
sequence_matrix = pad_sequences(sequence, maxlen=max_len)

# Create a Sequential model
model = Sequential()

# Add an Embedding layer
embedding_dim = 50  # Dimension of word embeddings
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

# Add an LSTM layer
lstm_units = 64  # Number of LSTM units
model.add(LSTM(units=lstm_units))

# Add a Dense (fully connected) layer for output
num_classes = 2  # Number of sentiment classes (positive and negative)
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 50)            50000     
                                                                 
 lstm_5 (LSTM)               (None, 64)                29440     
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 79570 (310.82 KB)
Trainable params: 79570 (310.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [102]:
# Train the model
model.fit(
    sequence_matrix,
    Y_train,
    batch_size=128, epochs=10,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
8/8 [==============================] - 1s 92ms/step - loss: 0.0786 - accuracy: 0.9715 - val_loss: 1.2450 - val_accuracy: 0.7087
Epoch 2/10
8/8 [==============================] - 1s 79ms/step - loss: 0.0654 - accuracy: 0.9764 - val_loss: 1.3991 - val_accuracy: 0.6969


In [103]:
# Evaluate the model on the test set
# loss, accuracy = model.evaluate(sequence_matrix_test, Y_test)
# print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

In [104]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len)

In [105]:
display(len(test_sequences_matrix))
display(len(Y_test))

545

545

In [106]:
loss, accr = model.evaluate(test_sequences_matrix, Y_test)

18/18 [==============================] - 0s 5ms/step - loss: 1.4187 - accuracy: 0.6972


In [107]:
print(f'Test loss: {loss:.4f}, Test accuracy: {accr:.4f}')

Test loss: 1.4187, Test accuracy: 0.6972
